__CLICK TO ZOOM MANDELBROT FRACTAL INTERACTIVE NAVIGATOR__
- __Left click to zoom In,__  
- __Right click to zoom Out.__  
- Figure is resizable (__bottom right drag__), but depending on your hardware it could get too slow
  - 1000x1000 pixels runs ok, meaning a million calculations (* max_iter) each draw
  - There's a glitch on the dpi getter so resize first -_ONCE_, then navigate later is recommended.

# Speed calculations by using a C compiled function
## Use a binary
 - Called mandelbrot.cpython-37m-x86_64-linux-gnu.so  
 - Compile it by running the setup.py pointing to mandelbrot.pyx file  
Then just import it:

In [1]:
from mandelbrot import mandelbrot

## Or compile it on the fly using cython

In [2]:
%load_ext cython

In [3]:
%%cython
cpdef halfGridMandelbrot( int[:,::1] itBreak, float left, float dx, int rx,
                                      float bottom, float dy, int ry, int maxIter):
    cdef complex z, c
    cdef int x, y, it
    
    cdef float dx2 = 2 * dx
    cdef float dy2 = 2 * dy
    
    c = left + dx
    for x in range(1, rx, 2):
        c.imag = bottom
        for y in range(0, ry):
            #print('fila impar',x,' todos col',y)
            z = c
            for it in range(maxIter):
                if z.real**2+z.imag**2 >= 4:
                    break
                else:
                    z = z*z + c
            itBreak[y,x] = it
            #print('c[{},{}]={}\tit={}'.format(x,y,c,it))
            c += 1j* dy
        c += dx2
            
    c = left
    for x in range(0, rx, 2):
        c.imag = bottom + dy
        for y in range(1, ry, 2):
            #print('fila par', x, 'col impar',y)
            z = c 
            for it in range(maxIter):
                if z.real**2+z.imag**2 >= 4:
                    break
                else:
                    z = z*z + c
            itBreak[y,x] = it
            #print('c[{},{}]={}\tit={}'.format(x,y,c,it))
            c += 1j* dy2
        c += dx2  

## Small speed gains
Disabling both autosave and 'Settings>Save Widget State Automatically' also helps

# Import
Check https://github.com/matplotlib/ipympl for installation, but `!pip install ipympl matplotlib numpy Cython` is a start for troubleshooting.  
Also there's a tk and Qt5 backend versions on the repo on separate py files

In [4]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

## If starting over without reseting the whole kernel

In [5]:
plt.close()

# This is it

In [6]:
def halfZoomBounds(left, right, bottom, top, rx, ry, exd, eyd):
    x = np.linspace(left, right, rx)
    y = np.linspace(bottom, top, ry)
    
    nix = np.argmin(np.abs(x-exd))
    niy = np.argmin(np.abs(y-eyd))
    
    nlix = nix-int(rx/4)
    nrix = nix+int(rx/4)
    nbiy = niy-int(ry/4)
    ntiy = niy+int(ry/4)
    
    newleft  = x[nlix]
    newright = x[nrix]
    newbottom= y[nbiy]
    newtop   = y[ntiy]
    
    return newleft, newright, newbottom, newtop, rx, ry, nlix, nrix, nbiy, ntiy

In [7]:
def correct4divisibility(x):
    r = x%4
    if r==0: #%100%4
        return x
    elif r<3:
        return x-r
    else:
        return x+1

In [9]:
def button_press(event):
    global c, coords, pendingDraw
    #print(event.name, 'I', c, pendingDraw, sep=',', end='; ')
    ax = event.inaxes 
    if event.button == 1:
        #print('# deal with zoom in')        
        left, right, bottom, top, rx, ry, nlix, nrix, nbiy, ntiy = halfZoomBounds(*coords[c],event.xdata,event.ydata)
        c+=1
        coords[c] = [left, right, bottom, top, rx, ry ]
        ax.set_xlim(left,right)
        ax.set_ylim(bottom,top)
        # Here you can choose from drawing directly (A) or show the zooming then redrawing (B)
        # A
        halfMandraw(ax, left, right, bottom, top, rx, ry , nlix, nrix, nbiy, ntiy)
        pendingDraw=False
        # B
        #pendingDraw=True
        ft.set_text('zoom In {} times, {:.1E} pixels calculated'.format(c, rx*ry*3/4))
    elif event.button == 3:
        #print('# deal with zoom out')
        if c==0:
            #print('Already initial image, not zooming out')
            ft.set_text('Already initial image, not zooming out')
            return
        c-=1
        left, right, bottom, top, rx, ry = coords[c]
        ax.set_xlim(left,right)
        ax.set_ylim(bottom,top)
        ft.set_text('zoom Out to {}'.format(c))
        pendingDraw=False
    else:
        # deal with something that should never happen
        print('wtf!')
    #plt.ion()
    #print('done!')
    #print(event.name, 'F', c, pendingDraw, sep=',', end='; ')

def resize(event):
    global coords, pendingDraw
    #print(event.name, 'I', c, pendingDraw, sep=',', end='; ')
    rx, ry = get_ax_wh(fig, ax)
    ft.set_text('Resize w{}, h{}, dpi{}'.format( rx, ry, fig.dpi))
    coords[c][4]=correct4divisibility(rx)
    coords[c][5]=correct4divisibility(ry)
    pendingDraw=True
    #print(event.name, 'F', c, pendingDraw, sep=',', end='; ')

def draw(event):
    global pendingDraw
    #print('pre draw', 'I', c, pendingDraw, sep=',', end='; ')
    if pendingDraw:
        #print(event.name, c, sep=',', end='; ')
        mandraw(ax, *coords[c] )
    else:
        pass
        #print(event.name, 'avoided')
    #print('post draw', 'F', c, pendingDraw, sep=',', end='; ')

def get_ax_wh(fig,ax):
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    width, height = int(bbox.width*fig.dpi), int(bbox.height*fig.dpi)    
    return width, height

def mandraw(ax, left, right, bottom, top, rx, ry ):
    global pendingDraw, b
    #print('pre mandraw', 'I', c, pendingDraw, sep=',', end='; ')
    dx = (right - left ) / rx
    dy = (top - bottom) / ry
    z = np.zeros((ry,rx), dtype=np.int32)
    b = z
    mandelbrot( z, left, dx, rx, bottom, dy, ry, maxIter) 
    ax.pcolorfast( (left,right), (bottom,top), z, cmap='terrain', vmin=0, vmax=maxIter)
    pendingDraw=False
    #print('post mandraw', 'F', c, pendingDraw, sep=',', end=';\n')
    
def halfMandraw(ax, left, right, bottom, top, rx, ry , nlix, nrix, nbiy, ntiy):
    global pendingDraw, b
    #print('pre mandraw', 'I', c, pendingDraw, sep=',', end='; ')
    dx = (right - left ) / rx
    dy = (top - bottom) / ry
    z = np.ones((ry,rx), dtype=np.int32)
    halfGridMandelbrot( z, left, dx, rx, bottom, dy, ry, maxIter)
    
    #print('nlix, nrix, nbiy, ntiy',nlix, nrix, nbiy, ntiy)
    
    for i,vi in enumerate(range( nlix, nrix)):
        for j,vj in enumerate(range( nbiy, ntiy)): 
            #print(i,' ',j)
            z[j*2,i*2] = b[vj,vi]
    
    ax.pcolorfast( (left,right), (bottom,top), z, cmap='terrain', vmin=0, vmax=maxIter)
    b=z
    pendingDraw=False
    #print('post mandraw', 'F', c, pendingDraw, sep=',', end=';\n')
    
# init plot
fig, ax = plt.subplots()
fig.set_size_inches(600/fig.dpi,600/fig.dpi)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

# connect events
ids=dict()
#ids[]=fig.canvas.mpl_connect('',)
ids['draw'] = fig.canvas.mpl_connect('draw_event', draw) 
ids['resize'] = fig.canvas.mpl_connect('resize_event', resize)
ids['button_press'] = fig.canvas.mpl_connect('button_press_event', button_press)

# Initial data
left, right = np.float64((-2.0, 0.66))
bottom, top = np.float64((-1.4, 1.4))
rx, ry = correct4divisibility(16), correct4divisibility(24)
b = np.zeros((ry,rx), dtype=np.int32)
coords=dict()
c=0
coords[c] = [left, right, bottom, top, rx, ry ] #print(c,coords[c])
pendingDraw=False
maxIter=63

# Initial plot
ft = fig.text(0.5,0.9,'w{}, h{}, dpi{}'.format( *get_ax_wh(fig,ax), fig.dpi))
mandraw(ax, left, right, bottom, top, rx, ry )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Not seeing enough colors (mainly inside the fractal)?
Increase the max iteration parameter, beware it also slows down.

In [11]:
from ipywidgets import BoundedIntText
bit = BoundedIntText( value=maxIter, min=1, max=256*2, step=10, description='Upper bound cut off iteration number:',
    disabled=False, style = {'description_width': 'initial'} )
def change_max_iter(change):
    global maxIter, pendingDraw
    if isinstance(change.new,int):
        maxIter = change.new
        pendingDraw=True
        fig.canvas.send_event('draw')
        #print(maxIter, pendingDraw, c, pendingDraw, sep=',', end=';\n')
bit.observe(change_max_iter)
bit

BoundedIntText(value=163, description='Upper bound cut off iteration number:', max=512, min=1, step=10, style=…

In [ ]:
def fsize():
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    axwidth, axheight = bbox.width, bbox.height
    axwidth *= fig.dpi
    axheight *= fig.dpi
    
    bbox = fig.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    figwidth, figheight = bbox.width, bbox.height
    figwidth *= fig.dpi
    figheight *= fig.dpi
    
    return axwidth, axheight, figwidth, figheight, fig.dpi, axwidth/figwidth, axheight/figheight

fsize()